<a href="https://colab.research.google.com/github/llaume974/Market-risk-measure-with-non-parametric-VaR/blob/main/Projet_market_risk_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part A

###Importation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pywt

In [ ]:
#load the dataset
file_path = '/content/Natixis stock (dataset TD1&2).txt'
natixis_data = pd.read_csv(file_path, delimiter='\t')
natixis_data.columns = ['Date', 'Price']

#convert 'Date' column to datetime format with the correct format
natixis_data['Date'] = pd.to_datetime(natixis_data['Date'], format='%d/%m/%Y')

#replace commas with periods in the 'Price' column
natixis_data['Price'] = natixis_data['Price'].str.replace(',', '.')

#convert 'Price' column to a numeric type
natixis_data['Price'] = pd.to_numeric(natixis_data['Price'], errors='coerce')

#calculate daily returns
natixis_data['Return'] = natixis_data['Price'].pct_change()

#split data in gains and loses
gains = natixis_data['Return'][natixis_data['Return'] > 0]
losses = -natixis_data['Return'][natixis_data['Return'] < 0]

#filter the dataset for the specified periods
data_filtered1 = natixis_data[(natixis_data['Date'] >= '2015-01-01') & (natixis_data['Date'] <= '2016-12-31')]
data_filtered2 = natixis_data[(natixis_data['Date'] >= '2017-01-01') & (natixis_data['Date'] <= '2018-12-31')]


##**A.**

From the time series of the daily prices of the stock Natixis between January 2015 and December 2016, provided with TD1, estimate a historical VaR on price returns at a one-day horizon for a given probability level (this probability is a parameter which must be changed easily). You must base your VaR on a non-parametric distribution


###Define functions

In [ ]:
def empirical_var(data, alpha, bandwidth):
    # Clean the data by removing NaNs and infinities
    cleaned_data = data[~np.isnan(data["Return"]) & ~np.isinf(data["Return"])]

    # Extract the returns after cleaning
    returns = cleaned_data["Return"].values
    n = len(returns)

    # Create an array of return values over which to evaluate the KDE
    return_values = np.linspace(returns.min(), returns.max(), 1000)

    # Calculate the kernel density estimate using the biweight kernel
    kde_values = np.zeros_like(return_values)
    for i in range(len(return_values)):
        u = (return_values[i] - returns) / bandwidth
        kernel_values = (15/16) * (1 - u**2)**2 * (np.abs(u) <= 1)
        kde_values[i] = kernel_values.sum() / (bandwidth * n)

    # Compute the cumulative distribution function (CDF) from the KDE values
    cdf_values = np.cumsum(kde_values) / np.sum(kde_values)

    # Interpolate to find the VaR corresponding to the given alpha level
    var = np.interp(1-alpha, cdf_values, return_values)

    return var

###VaR calculation

In [ ]:
#Scott empirical bandwidth
bandwidth = (4 / (3 * len(data_filtered1["Return"]))) ** 0.2 * np.std(data_filtered1["Return"])
proba = 0.95


var = empirical_var(data_filtered1, proba, bandwidth)
print("Historical VaR computed with biweight kernel function:", var)

Historical VaR computed with biweight kernel function: -0.03793175395568973


##**B.**

Which proportion of price returns between January 2017 and December 2018 exceed the VaR
threshold defined in the previous question? Do you validate the choice of this non-parametric VaR?

In [ ]:
#count the number of returns that exceed the VaR threshold
exceedances = (data_filtered2['Return'] < var).sum()

#calculate the proportion of exceedances
total_observations = len(data_filtered2['Return'])
proportion_exceedances = exceedances / total_observations

print("Proportion of Exceedances:", proportion_exceedances)

Proportion of Exceedances: 0.01568627450980392


#Part B

Calculate the expected shortfall for the VaR calculated in question A. How is the result, compared to
the VaR?

In [ ]:
#filter out returns worse than the VaR
returns_worse_than_var = data_filtered2[data_filtered2['Return'] < var]['Return']

#calculate the ES
if not returns_worse_than_var.empty:
    expected_shortfall = returns_worse_than_var.mean()
else:
    expected_shortfall = np.nan

print("Expected Shortfall:", expected_shortfall)

Expected Shortfall: -0.05090042507108643


#Part C

##A.

Estimate the GEV parameters for the two tails of the distribution of returns, using the estimator of
Pickands. What can you conclude about the nature of the extreme gains and losses?

###Functions

In [ ]:
def pickands_estimator(data):

    #fraction of data to use
    sorted_data = np.sort(data)[::-1]
    k = int(0.1 * len(sorted_data))
    if k < 1:
        raise ValueError("Not enough data to apply Pickands estimator.")

    #calculate the Pickands estimator for the GEV xi parameter
    xi = (1 / np.log(2)) * np.log((sorted_data[-k] - sorted_data[-2*k]) / (sorted_data[-2*k] - sorted_data[-4*k]))

    #calculate mu and sigma parameters as the mean and standard deviation of the top k data points
    mu = np.mean(sorted_data[:k])
    sigma = np.std(sorted_data[:k])

    return {'xi': xi, 'mu': mu, 'sigma': sigma}

###Results

In [ ]:
#call and display of Pickands estimator
gev_params_gains = pickands_estimator(gains)
gev_params_losses = pickands_estimator(losses)

#analyse parameters
print("GEV Parameters for Gains:", gev_params_gains)
print("GEV Parameters for Losses:", gev_params_losses)

GEV Parameters for Gains: {'xi': -1.017904110288953, 'mu': 0.044886823821665124, 'sigma': 0.012584666884058687}
GEV Parameters for Losses: {'xi': -0.8150241887276759, 'mu': 0.04646321776268792, 'sigma': 0.020089520576226726}


##B.

Calculate the value at risk based on EVT for various confidence levels, with the assumption of iid
returns.

###Functions

In [ ]:
def calculate_var_from_gev(data, confidence_level, gev_params):
    n = len(data)
    k = int(0.1 * n)

    #fraction of data to use
    if k < 1:
        raise ValueError("Not enough data to apply Pickands estimator.")
    if 4*k >= n:
        raise ValueError("k is too large for the dataset provided.")

    xi = gev_params['xi']
    mu = gev_params['mu']
    sigma = gev_params['sigma']

    #calculate the necessary quantiles
    x_n_k = data[-k]
    x_n_2k = data[-2*k]

    #calculate the VaR
    term1 = (k / (n * (1 - confidence_level))) ** xi
    var = x_n_k + (x_n_k * x_n_2k * ((term1 - 1)) / (1 - 2 ** (-xi)))

    return var

###Results

In [ ]:
#confidence levels to calculate VaR
confidence_levels = [0.95, 0.99]

# Calculate VaR for the specified confidence levels using the gains GEV parameters
vars_gains = {conf: calculate_var_from_gev(np.sort(gains), conf, gev_params_gains) for conf in confidence_levels}

# Calculate VaR for the specified confidence levels using the losses GEV parameters
vars_losses = {conf: calculate_var_from_gev(np.sort(losses), conf, gev_params_losses) for conf in confidence_levels}

print("Calculated VaR for gains :", vars_gains)
print("Calculated VaR for losses :", vars_losses)

Calculated VaR for gains : {0.95: 0.03149852778575379, 0.99: 0.03180787165738903}
Calculated VaR for losses : {0.95: 0.03212828869324476, 0.99: 0.032507172832029814}


#Part D

 Estimate all the parameters of the model of Almgren and Chriss. Is this model well specified?


In the framework of Almgren and Chriss, what is your liquidation strategy (we recall that you can
only make transactions once every hour).

###Importation

In [ ]:
#load the dataset
file_path = '/content/Dataset TD4.xlsx'
td4_data = pd.read_excel(file_path)

td4_data.columns = ['transaction_date', 'bid_ask_spread', 'volume', 'sign', 'price_before']

#date to index
td4_data['transaction_index'] = np.arange(len(td4_data))

#handle NaN values
td4_data['volume'].fillna(0, inplace=True)

###Functions

In [ ]:
#function g(v)
def g(v, gamma):
    return gamma * v

#function h(n_k/τ)
def h(n_k, xi, eta, tau):
    return xi * np.sign(n_k) + eta * (n_k / tau)

def total_cost(df, gamma, eta, tau):
    #handle NaN values in volume by replacing
    df['volume'].fillna(0, inplace=True)

    #price change
    price_changes = df['price_before'].diff().fillna(0)

    #v_k and n_k
    v_k = df['volume'] / tau
    n_k = df['volume']

    #g(v) for all transactions
    linear_costs = g(v_k, gamma)

    #h(n_k/τ) for all transactions
    quadratic_costs = h(n_k, df['bid_ask_spread'], eta, tau)

    #price impact cost for all transactions
    price_impact_costs = 0.5 * gamma * price_changes ** 2

    #total cost
    total_costs = tau * linear_costs + n_k * quadratic_costs + price_impact_costs

    return total_costs.sum()

###Results

In [ ]:
tau = 1
gamma = 0.1
eta = 0.1
learning_rate = 0.001
iterations = 300
epsilon = 1e-5

#initialize previous cost
previous_cost = float('inf')

#gradient descent
for iteration in range(iterations):
    #current cost for convergence check
    current_cost = total_cost(td4_data, gamma, eta, tau)

    #if change in cost is below tolerance, stop the optimization
    if abs(previous_cost - current_cost) < epsilon:
        print(f"Convergence reached at iteration {iteration}.")
        break

    previous_cost = current_cost

    #calculation of gradients by finite difference
    gamma_gradient = (total_cost(td4_data, gamma + epsilon, eta, tau) - total_cost(td4_data, gamma, eta, tau)) / epsilon
    eta_gradient = (total_cost(td4_data, gamma, eta + epsilon, tau) - total_cost(td4_data, gamma, eta, tau)) / epsilon

    gamma = max(0, gamma - learning_rate * gamma_gradient)
    eta = max(0, eta - learning_rate * eta_gradient)

# Print the optimized parameters
print("Total Cost :", current_cost)
print("Optimized gamma:", gamma)
print("Optimized eta:", eta)

Convergence reached at iteration 2.
Total Cost : 1505.5773
Optimized gamma: 0
Optimized eta: 0


#Part E

##A.

Determine the correlation matrix at each scale using wavelets. Determine the volatility vector with
a scaling thanks to the Hurst exponents. From the correlation matrix and the volatility vector, calculate
the covariance matrix and conclude about the volatility of the portfolio.

###Importation

In [ ]:
#load the dataset
file_path = '/content/Dataset TD5.xlsx'
td5_data = pd.read_excel(file_path)

# Dropping unnecessary columns and rows
td5_data = td5_data.drop(columns=['Unnamed: 3', 'Unnamed: 7'], axis=1)
td5_data = td5_data.drop([0, 1], axis=0).reset_index(drop=True)

# Renaming the columns for clarity
column_names = ['Date_GBPEUR', 'HIGH_GBPEUR', 'LOW_GBPEUR', 'Date_SEKEUR', 'HIGH_SEKEUR', 'LOW_SEKEUR', 'Date_CADEUR', 'HIGH_CADEUR', 'LOW_CADEUR']
td5_data.columns = column_names

#average of High and Low for each FX rate
td5_data['AVG_GBPEUR'] = td5_data[['HIGH_GBPEUR', 'LOW_GBPEUR']].mean(axis=1)
td5_data['AVG_SEKEUR'] = td5_data[['HIGH_SEKEUR', 'LOW_SEKEUR']].mean(axis=1)
td5_data['AVG_CADEUR'] = td5_data[['HIGH_CADEUR', 'LOW_CADEUR']].mean(axis=1)

td5_data[['Date']] = td5_data[['Date_GBPEUR']]

#returns for each FX rate and set the Date as the index
td5_data['RTN_GBPEUR'] = td5_data['AVG_GBPEUR'].pct_change()
td5_data['RTN_SEKEUR'] = td5_data['AVG_SEKEUR'].pct_change()
td5_data['RTN_CADEUR'] = td5_data['AVG_CADEUR'].pct_change()

# Select only the average rates for analysis
fx_rates = td5_data[['Date', 'AVG_GBPEUR', 'AVG_SEKEUR', 'AVG_CADEUR']]
fx_rates.set_index('Date', inplace=True)

# Calculating the portfolio return as the weighted average of the FX rates returns
weights = np.array([1/3, 1/3, 1/3])  # Equal weights

fx_returns = td5_data[['Date', 'RTN_GBPEUR', 'RTN_SEKEUR', 'RTN_CADEUR']]
fx_returns.set_index('Date', inplace=True)

# Calculate the portfolio return with equal weights
fx_returns['Portfolio_Return'] = (fx_returns[['RTN_GBPEUR', 'RTN_SEKEUR', 'RTN_CADEUR']] * weights).sum(axis=1)

<ipython-input-196-b3b480ca77b2>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fx_returns['Portfolio_Return'] = (fx_returns[['RTN_GBPEUR', 'RTN_SEKEUR', 'RTN_CADEUR']] * weights).sum(axis=1)


###Functions

In [ ]:
#rescaling the data
def resample_data(dataframe, freq):
    if freq == '1H':
        filtered = dataframe[dataframe.index.minute == 0]
        resampled = filtered.resample(freq).mean()
    else:
        resampled = dataframe.resample(freq).mean()

    return resampled

def wavelet_correlation(dataframe, wavelet='haar', mode='symmetric', level=1):
    transformed_columns = []

    for column in dataframe.columns:
        coeffs = pywt.wavedec(dataframe[column].dropna(), wavelet=wavelet, mode=mode, level=level)
        approximation = coeffs[0]

        #adjust the length of the approximation
        if len(approximation) < len(dataframe):
            # If the approximation is shorter, extend it with NaNs
            approximation = np.pad(approximation, (0, len(dataframe) - len(approximation)), 'constant', constant_values=np.nan)
        elif len(approximation) > len(dataframe):
            # If the approximation is longer, truncate it
            approximation = approximation[:len(dataframe)]

        transformed_column_name = f'WT_{column}'
        dataframe[transformed_column_name] = approximation
        transformed_columns.append(transformed_column_name)

    #calculate the correlation matrix using the correct column names
    correlation_matrix = dataframe[transformed_columns].corr()
    return correlation_matrix

#hurst exponent
def hurst_exponent(ts):
    N = len(ts)
    M2 = np.sum([(ts[i] - ts[i - 1]) ** 2 for i in range(1, N)]) / N
    M2_prime = np.sum([(ts[2 * i] - ts[2 * (i - 1)]) ** 2 for i in range(1, N // 2)]) / (N / 2)

    if M2 == 0:
        return np.nan

    H_hat = 0.5 * np.log2(M2_prime / M2)
    return H_hat

#covariance matrix from correlation matrix and volatilities
def calculate_covariance_matrix(correlation_matrix, volatilities):
    volatilities_diag = np.diag(volatilities)

    covariance_matrix = volatilities_diag.dot(correlation_matrix).dot(volatilities_diag)
    return covariance_matrix


#portfolio volatility
def portfolio_volatility(covariance_matrix, weights):
    return np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

###Correlation Matrix

In [ ]:
#time scales for resampling
scales = ['15min', '1H', '1D', '1W']

correlation_matrices = {}

for scale in scales:
    #resample data
    resampled_data = resample_data(fx_rates, scale)

    #correlation matrix
    correlation_matrix = wavelet_correlation(resampled_data)
    correlation_matrices[scale] = correlation_matrix

    print(f"Correlation Matrix for {scale} scale:\n{correlation_matrices[scale]}\n")


Correlation Matrix for 15min scale:
               WT_AVG_GBPEUR  WT_AVG_SEKEUR  WT_AVG_CADEUR
WT_AVG_GBPEUR       1.000000       0.814763      -0.226994
WT_AVG_SEKEUR       0.814763       1.000000      -0.135315
WT_AVG_CADEUR      -0.226994      -0.135315       1.000000

Correlation Matrix for 1H scale:
               WT_AVG_GBPEUR  WT_AVG_SEKEUR  WT_AVG_CADEUR
WT_AVG_GBPEUR       1.000000       0.815376      -0.227289
WT_AVG_SEKEUR       0.815376       1.000000      -0.136208
WT_AVG_CADEUR      -0.227289      -0.136208       1.000000

Correlation Matrix for 1D scale:
               WT_AVG_GBPEUR  WT_AVG_SEKEUR  WT_AVG_CADEUR
WT_AVG_GBPEUR       1.000000       0.822674      -0.256434
WT_AVG_SEKEUR       0.822674       1.000000      -0.178265
WT_AVG_CADEUR      -0.256434      -0.178265       1.000000

Correlation Matrix for 1W scale:
               WT_AVG_GBPEUR  WT_AVG_SEKEUR  WT_AVG_CADEUR
WT_AVG_GBPEUR       1.000000       0.859594      -0.347460
WT_AVG_SEKEUR       0.859594       1

###Volatility with Hurst exposent

In [ ]:
#hurst exponent
hurst_exponents = fx_rates.apply(hurst_exponent)

#daily volatility
daily_volatilities = fx_rates.diff().apply(np.std)

#volatilities by the Hurst exponent
scaled_volatilities = daily_volatilities * hurst_exponents

#results
print("Hurst Exponents:\n", hurst_exponents)
print("Daily Volatilities:\n", daily_volatilities)
print("Scaled Volatilities:\n", scaled_volatilities)

Hurst Exponents:
 AVG_GBPEUR    0.671373
AVG_SEKEUR    0.654591
AVG_CADEUR    0.655240
dtype: float64
Daily Volatilities:
 AVG_GBPEUR    0.000775
AVG_SEKEUR    0.000035
AVG_CADEUR    0.000348
dtype: float64
Scaled Volatilities:
 AVG_GBPEUR    0.000521
AVG_SEKEUR    0.000023
AVG_CADEUR    0.000228
dtype: float64


Covariance Matrix

In [ ]:
#covariance matrices
covariance_matrices = {}
for scale in scales:
    #correlation matrix
    correlation_matrix = correlation_matrices[scale]

    covariance_matrix = calculate_covariance_matrix(correlation_matrix, scaled_volatilities)
    covariance_matrices[scale] = covariance_matrix

#weights of each assets
weights = np.array([1/3, 1/3, 1/3])

#portfolio volatility for each scale
portfolio_volatilities = {}
for scale, covariance_matrix in covariance_matrices.items():
    vol = portfolio_volatility(covariance_matrix, weights)
    portfolio_volatilities[scale] = vol

#portfolio volatilities
for scale, vol in portfolio_volatilities.items():
    print(f"Portfolio Volatility for {scale} scale: {vol}")

Portfolio Volatility for 15min scale: 0.00017875819532757874
Portfolio Volatility for 1H scale: 0.00017873807694286864
Portfolio Volatility for 1D scale: 0.00017649201277009075
Portfolio Volatility for 1W scale: 0.00016940297348471312


##B.

Determine the volatility directly from the series of prices of the portfolio and justify your
methodological choices (for example with overlapping returns or not).

In [ ]:
#time scales
time_scales = [(15,60,1440,10080),("15min","1H","1D","1W")]
i=0
volatility_dict = {}

for scale in time_scales[0]:
    #resample the portfolio returns data for each time scale and calculate the mean
    resampled_returns = fx_returns['Portfolio_Return'].resample(f'{scale}T').mean()

    #standard deviation (volatility) for each time scale
    volatility_corrected = resampled_returns.std()
    volatility_dict[scale] = volatility_corrected

#portfolio volatilities
for scale, vol in volatility_dict.items():
    print("Portfolio Volatility for ", time_scales[1][i], f"scale: {vol}")
    i+=1

Portfolio Volatility for  15min scale: 0.00034131077611115635
Portfolio Volatility for  1H scale: 0.0001965199615230007
Portfolio Volatility for  1D scale: 6.453290289445081e-05
Portfolio Volatility for  1W scale: 1.7611217930234627e-05
